<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=252ab83fb9660a1248a4f4fe414711456ce59894ec80c869be849762da92f25d
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-25 10:22:54
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  400.00
Current:  1.42 C
-------------------
Today PnL: -23.70 K (-0.17%)
Current PnL: -23.58 L (-15.45%)
CY Booked + Current PnL: -9.69 L (-6.35%)
-------------------
Total profit:  1.30 L
Total loss:  -24.88 L
-------------------
Total Booked + Current PnL: 17.29 L (13.76%)
Total Booked PnL: 40.87 L (32.53%)
Curr Year Booked PnL: 13.89 L (9.77%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 88.99 L (62.58%)
Deployed:  1.26 C
Current:  1.42 C
CAGR/XIRR %: 7.05%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.00,0.20,-14.04,16.45,0.09,14250.0,-14153.0,86624.0,101.07,50.0,M-SC,3.98,253.0,-0.99,0.61,13.80,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,-0.20,-9.51,10.54,0.02,16648.0,-16608.0,157951.0,141.07,69.0,M-SC,6.66,234.0,-1.00,1.11,37.39,OX40N,NTT,PAINTS
35,ICICIGI,2252.93,0.51,7.11,12.21,20.19,22798.0,12396.0,186716.0,-16.02,51.0,X-MC,7.60,68.0,0.54,1.31,23.30,X40,ATH,INSURANCE
43,ITC,452.00,0.16,-1.24,11.83,10.44,23382.0,-2484.0,197654.0,-41.15,44.0,X-LC,0.82,5.0,-0.11,1.39,4.25,X40,NTT,FMCG
51,MASFIN,398.61,0.55,-4.68,28.05,22.05,26196.0,-4590.0,93390.0,-17.80,51.0,H-SC,7.37,164.0,-0.18,0.66,35.75,XR,ATH,FINANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,ROUTE,2227.21,4.29,-49.18,232.64,69.05,157323.0,-65441.0,67625.0,-58.98,38.0,H-SC,23.54,140.0,-0.42,0.48,4.29,SR,ATH,IT
61,REPCOHOME,880.00,2.76,-10.59,112.30,89.82,287669.0,-30344.0,256161.0,-55.29,50.0,H-SC,10.43,160.0,-0.11,1.80,35.53,XY24,NTT,FINANCE
56,QUESS,424.00,2.45,-27.39,96.72,42.84,45653.0,-17805.0,47201.0,-52.62,41.0,X-SC,37.35,83.0,-0.39,0.33,2.71,XY24,NTT,MISC
64,SFL,1287.00,2.10,-39.94,110.85,26.63,174570.0,-104735.0,157483.0,19.19,33.0,M-SC,4.50,239.0,-0.60,1.11,2.10,XY24,NTT,MISC
18,COFFEEDAY,80.00,1.93,-38.30,119.24,35.27,83541.0,-43488.0,70061.0,-51.71,36.0,L-SC,17.78,270.0,-0.52,0.49,71.48,XR,NTT,HOTELS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,AWL,485.00,-4.66,-16.24,83.26,53.50,210432.0,-49010.0,252741.0,-59.42,42.0,X-MC,4.09,58.0,-0.23,1.78,12.26,XY24,NTT,FMCG
34,HONAUT,58357.33,-2.59,-13.24,65.20,43.32,92128.0,-21572.0,141300.0,-25.47,46.0,X-SC,1.97,90.0,-0.23,0.99,8.49,X40N,ATH,ELECTRICAL
57,RAJESHEXPO,518.00,-2.31,-63.87,176.99,0.07,87713.0,-87619.0,49558.0,1716.92,53.0,L-SC,1.54,268.0,-1.00,0.35,23.25,OX40N,NTT,JEWELLERY
3,ACC,3906.00,-2.23,-21.37,108.77,64.15,203509.0,-50851.0,187100.0,-54.13,54.0,X-SC,1.63,82.0,-0.25,1.32,4.95,XY24,BTT,CEMENT
68,SONACOMS,806.63,-1.63,-13.93,62.12,39.54,54089.0,-14088.0,87071.0,-32.63,60.0,M-SC,6.67,220.0,-0.26,0.61,22.78,AR,ATH,AUTO


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-0.04,-0.89,108.57,106.72,132720.0,-1092.0,122244.0,-51.01,32.0,H-SC,3.69,139.0,-0.01,0.86,22.29,AR,ATH,MISC
85,WIPRO,420.00,-0.75,0.66,71.14,72.27,121336.0,1112.0,170560.0,-13.79,53.0,M-LC,4.55,99.0,0.01,1.20,6.87,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.27,-5.53,120.09,107.91,167494.0,-8170.0,139474.0,-22.98,32.0,M-SC,10.49,216.0,-0.05,0.98,0.00,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-0.20,-9.51,10.54,0.02,16648.0,-16608.0,157951.0,141.07,69.0,M-SC,6.66,234.0,-1.00,1.11,37.39,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,0.20,-14.04,16.45,0.09,14250.0,-14153.0,86624.0,101.07,50.0,M-SC,3.98,253.0,-0.99,0.61,13.80,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-0.84,-23.23,47.80,13.47,98963.0,-62631.0,207036.0,-68.51,25.0,H-SC,1.38,158.0,-0.63,1.46,5.27,XY24,NTT,PAINTS
44,JCHAC,2282.00,-0.35,-23.47,30.65,-0.01,29979.0,-29995.0,97810.0,19921.74,55.0,M-SC,1.67,233.0,-1.00,0.69,10.75,OX40N,NTT,AC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.0,-0.75,0.66,71.14,72.27,121336.0,1112.0,170560.0,-13.79,53.0,M-LC,4.55,99.0,0.01,1.2,6.87,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,-0.75,0.66,71.14,72.27,121336.0,1112.0,170560.0,-13.79,53.0,M-LC,4.55,99.0,0.01,1.20,6.87,XR,NTT,IT
38,INDIAMART,4810.62,-0.04,-0.89,108.57,106.72,132720.0,-1092.0,122244.0,-51.01,32.0,H-SC,3.69,139.0,-0.01,0.86,22.29,AR,ATH,MISC
86,ZYDUSLIFE,1286.17,0.75,-1.55,37.76,35.63,77909.0,-3248.0,206326.0,-15.41,38.0,H-MC,4.28,119.0,-0.04,1.45,14.86,AR,ATH,PHARMA
25,FINCABLES,1641.55,-0.27,-5.53,120.09,107.91,167494.0,-8170.0,139474.0,-22.98,32.0,M-SC,10.49,216.0,-0.05,0.98,0.00,OX40N,ATH,CABLES
84,WHIRLPOOL,2270.00,-0.25,-5.43,92.36,81.92,100274.0,-6229.0,108569.0,-44.55,32.0,M-SC,4.51,236.0,-0.06,0.76,28.53,XR,NTT,DURABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,600.0,-0.49,-15.51,68.28,42.18,159461.0,-42870.0,233540.0,-24.30,21.0,X-LC,0.59,3.0,-0.27,1.64,0.00,XY24,NTT,AUTO
11,BATAINDIA,2096.0,-0.24,-35.28,108.91,35.20,90694.0,-45396.0,83274.0,9.17,25.0,X-SC,12.88,92.0,-0.50,0.59,0.29,X40,NTT,FOOTWEAR
60,RELAXO,1176.0,-0.47,-48.07,193.27,52.31,145702.0,-69772.0,75388.0,-44.73,26.0,X-SC,3.97,91.0,-0.48,0.53,0.91,X40N,NTT,FOOTWEAR
46,JSWINFRA,342.0,0.48,-8.93,26.53,15.23,48405.0,-17895.0,182452.0,-22.11,27.0,X-MC,6.63,66.0,-0.37,1.28,20.44,X40N,NTT,REALTY
27,GLAXO,3466.2,-1.37,-1.93,41.88,39.15,81850.0,-3844.0,195440.0,-21.29,28.0,X-MC,9.48,60.0,-0.05,1.37,26.51,X40,ATH,PHARMA


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,600.0,-0.49,-15.51,68.28,42.18,159461.0,-42870.0,233540.0,-24.30,21.0,X-LC,0.59,3.0,-0.27,1.64,0.00,XY24,NTT,AUTO
43,ITC,452.0,0.16,-1.24,11.83,10.44,23382.0,-2484.0,197654.0,-41.15,44.0,X-LC,0.82,5.0,-0.11,1.39,4.25,X40,NTT,FMCG
36,ICICIPRULI,790.0,0.63,1.33,29.25,30.97,52381.0,2344.0,179082.0,-22.52,49.0,X-MC,1.57,75.0,0.04,1.26,14.05,X40,ATH,INSURANCE
3,ACC,3906.0,-2.23,-21.37,108.77,64.15,203509.0,-50851.0,187100.0,-54.13,54.0,X-SC,1.63,82.0,-0.25,1.32,4.95,XY24,BTT,CEMENT
66,SIEMENS,4671.5,0.09,-14.91,47.51,25.51,75230.0,-27750.0,158345.0,-2.87,53.0,H-LC,1.87,49.0,-0.37,1.11,16.69,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,600.00,-0.49,-15.51,68.28,42.18,159461.0,-42870.0,233540.0,-24.30,21.0,X-LC,0.59,3.0,-0.27,1.64,0.00,XY24,NTT,AUTO
9,BAJAJHFL,181.50,0.02,-13.48,71.58,48.45,124176.0,-27027.0,173479.0,-19.66,32.0,X-MC,6.81,64.0,-0.22,1.22,0.24,X40N,ATH,FINANCE
11,BATAINDIA,2096.00,-0.24,-35.28,108.91,35.20,90694.0,-45396.0,83274.0,9.17,25.0,X-SC,12.88,92.0,-0.50,0.59,0.29,X40,NTT,FOOTWEAR
54,PAGEIND,51605.07,-0.37,-5.42,33.29,26.07,51553.0,-8880.0,154860.0,-30.31,34.0,X-MC,9.03,55.0,-0.17,1.09,0.39,X40,ATH,APPARELS
55,PGHH,17907.65,-0.44,-4.55,40.13,33.76,76923.0,-9135.0,191685.0,-32.64,33.0,X-MC,3.37,57.0,-0.12,1.35,0.41,X40,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,2.45,-27.39,96.72,42.84,45653.0,-17805.0,47201.0,-52.62,41.0,X-SC,37.35,83.0,-0.39,0.33,2.71,XY24,NTT,MISC
60,RELAXO,1176.00,-0.47,-48.07,193.27,52.31,145702.0,-69772.0,75388.0,-44.73,26.0,X-SC,3.97,91.0,-0.48,0.53,0.91,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.24,-35.28,108.91,35.20,90694.0,-45396.0,83274.0,9.17,25.0,X-SC,12.88,92.0,-0.50,0.59,0.29,X40,NTT,FOOTWEAR
52,MEDANTA,1486.00,-1.18,3.18,20.26,24.08,26537.0,4034.0,130984.0,-5.17,46.0,X-SC,4.95,89.0,0.15,0.92,23.42,XY24,NTT,HEALTHCARE
22,DIXON,18931.72,0.11,-5.69,28.92,21.58,38222.0,-7974.0,132165.0,-52.49,32.0,X-MC,6.83,56.0,-0.21,0.93,17.01,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,-0.26,-14.87,40.12,19.28,118154.0,-51454.0,294502.0,-25.89,60.0,X-LC,7.52,1.0,-0.44,2.07,8.87,X40,ATH,IT
41,INFY,2275.00,-0.90,4.43,48.31,54.87,157849.0,13854.0,326742.0,-18.52,57.0,X-LC,3.00,2.0,0.09,2.30,13.08,X40,BTT,IT
76,TMPV,600.00,-0.49,-15.51,68.28,42.18,159461.0,-42870.0,233540.0,-24.30,21.0,X-LC,0.59,3.0,-0.27,1.64,0.00,XY24,NTT,AUTO
82,VBL,671.64,0.38,-9.54,49.99,35.67,142819.0,-30146.0,285696.0,-20.30,42.0,X-LC,5.95,4.0,-0.21,2.01,2.99,X40N,ATH,FMCG
43,ITC,452.00,0.16,-1.24,11.83,10.44,23382.0,-2484.0,197654.0,-41.15,44.0,X-LC,0.82,5.0,-0.11,1.39,4.25,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,1.53,-11.91,106.23,81.67,87787.0,-11171.0,82639.0,7433.33,54.0,L-SC,18.82,271.0,-0.13,0.58,60.03,XR,NTT,CERAMICS
51,MASFIN,398.61,0.55,-4.68,28.05,22.05,26196.0,-4590.0,93390.0,-17.80,51.0,H-SC,7.37,164.0,-0.18,0.66,35.75,XR,ATH,FINANCE
70,SURYODAY,216.00,0.85,-19.14,52.17,23.03,75536.0,-34282.0,144789.0,57.89,49.0,H-SC,10.64,167.0,-0.45,1.02,43.66,XR,NTT,BANKS
39,INDIGOPNTS,1408.00,-0.20,-9.51,10.54,0.02,16648.0,-16608.0,157951.0,141.07,69.0,M-SC,6.66,234.0,-1.00,1.11,37.39,OX40N,NTT,PAINTS
35,ICICIGI,2252.93,0.51,7.11,12.21,20.19,22798.0,12396.0,186716.0,-16.02,51.0,X-MC,7.60,68.0,0.54,1.31,23.30,X40,ATH,INSURANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,-0.20,-9.51,10.54,0.02,16648.0,-16608.0,157951.0,141.07,69.0,M-SC,6.66,234.0,-1.00,1.11,37.39,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,0.84,1.48,19.18,20.95,49003.0,3722.0,255490.0,0.49,78.0,X-LC,13.43,31.0,0.08,1.80,37.64,X40,ATH,PAINTS
40,INDUSINDBK,1800.00,-0.34,-38.49,116.03,32.89,55105.0,-29714.0,47492.0,-686.82,58.0,L-MC,5.56,259.0,-0.54,0.33,30.79,XR,NTT,BANKS
12,BERGEPAINT,680.00,0.03,3.24,17.34,21.14,40697.0,7359.0,234698.0,-4.13,64.0,X-MC,3.92,74.0,0.18,1.65,32.29,XY24,NTT,PAINTS
79,UNITDSPR,1644.00,-0.03,5.11,14.90,20.77,36669.0,11954.0,246098.0,-3.99,59.0,X-MC,3.56,62.0,0.33,1.73,12.38,X40N,NTT,BREWERIES


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.73
1,25,44.65
2,50,75.53


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.63
MC    29.24
LC    25.13
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.97
X40      21.20
X40N     11.89
XR        9.64
XY25      9.07
AR        8.94
OX40N     7.65
X200      1.86
SR        1.00
MH        0.78
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.95
X-MC    22.80
X-LC    19.99
M-SC    12.02
X-SC     8.24
H-MC     4.73
H-LC     2.97
L-SC     1.42
M-MC     1.38
M-LC     1.20
L-LC     0.97
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.74,-5.80,40.01
IT,13.76,-15.59,75.26
FINANCE,9.49,-14.91,63.46
MISC,7.40,-23.33,73.85
PAINTS,6.02,-7.97,24.01
ELECTRICAL,5.91,-11.57,52.44
INSURANCE,4.45,0.05,35.17
PHARMA,3.86,-1.62,34.14
AUTO,3.33,-21.31,73.33


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3061181.0,21
AR,1298203.0,10
XR,1290235.0,13
X40,1024914.0,14
X40N,756177.0,9
OX40N,703573.0,10
XY25,352610.0,6
SR,278942.0,2
MH,73488.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3573298.0,25
M-SC,1348564.0,15
X-MC,1239959.0,16
X-LC,920380.0,11
X-SC,748667.0,8
H-MC,406694.0,3
L-SC,259041.0,3
H-LC,135059.0,2
M-LC,121336.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1217568.0      6
           AR         881899.0      5
           XR         792595.0      7
M-SC       XY24       786304.0      6
X-MC       X40        449370.0      7
X-LC       X40        398574.0      5
X-MC       XY24       359001.0      3
H-SC       OX40N      328806.0      4
X-SC       X40N       295998.0      3
M-SC       OX40N      287054.0      5
H-SC       SR         278942.0      2
X-SC       XY24       275699.0      3
X-MC       X40N       247472.0      4
X-LC       XY24       231654.0      2
H-MC       AR         215739.0      2
X-LC       X40N       212707.0      2
H-MC       XY24       190955.0      1
X-MC       XY25       184116.0      2
X-SC       X40        176970.0      2
L-SC       XR         171328.0      2
M-SC       XR         149871.0      2
           AR         125335.0      2
M-LC       XR         121336.0      1
L-SC       OX40N       87713.0      1
X-LC       XY25        77445.0      2
H-LC       AR          75230.0      1
H-SC       MH          73488.0      1
H-LC       X200        59829.0      1
L-MC       XR          55105.0      1
M-MC       XY25        49748.0      1
L-LC       XY25        41301.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
